In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse

In [19]:
endpoint_url = "http://localhost:7200/repositories/new_project"  # Cambia esto según tu configuración

def ejecutar_consulta(endpoint_url, consulta):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(consulta)
    sparql.setReturnFormat(JSON)
    resultados = sparql.query().convert()
    return resultados["results"]["bindings"]

## Previous queries

In [30]:
query_min_max = """
PREFIX ns: <http://example.org/ns#>

SELECT (MIN(?rating) AS ?minRating) (MAX(?rating) AS ?maxRating) (MIN(?count) AS ?minCount) (MAX(?count) AS ?maxCount)
WHERE {
    ?author ns:hasAvgRating ?rating .
    ?author ns:ratedBy ?count .
}"""

min_max = ejecutar_consulta(endpoint_url,query_min_max)
for k in min_max[0]:
    print(f'{k}: {float(min_max[0][k]['value'])}')

minRating: 0.0
maxRating: 5.0
minCount: 0.0
maxCount: 18532721.0


In [31]:
query_avg_popularity = f"""PREFIX ns: <http://example.org/ns#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT (AVG(?popularity) AS ?avgpopular)
    WHERE {{
        ?author rdf:type ns:Author .
        ?author ns:hasAvgRating ?AuthorRating .
        ?author ns:ratedBy ?count .
        BIND(((?AuthorRating - {float(min_max[0]['minRating']['value'])}) / ({float(min_max[0]['maxRating']['value'])} - {float(min_max[0]['minRating']['value'])}) * 0.5 + 
                (?count - {float(min_max[0]['minCount']['value'])}) / ({float(min_max[0]['maxCount']['value'])} - {float(min_max[0]['minCount']['value'])}) * 0.5) AS ?popularity)
    }}"""

avg_popularity = ejecutar_consulta(endpoint_url,query_avg_popularity)

In [35]:
popularity_threashold = float(avg_popularity[0]['avgpopular']['value'])

In [39]:
popularity_threashold

0.3943175919023493

## Find your perfect book

In [78]:
from SPARQLWrapper import SPARQLWrapper, JSON
import urllib.parse

# Función para pedir preferencias al usuario
def pedir_preferencias():
    genero = input("¿Qué género estás buscando? ").strip()
    longitud = input("¿Te gustan libros cortos/medios/largos? ").strip().lower()
    libro_reciente = input("¿Tienes una referencia de algún libro que hayas leído recientemente? ").strip()
    tipo_autor = input("¿Prefieres un autor famoso o poco conocido? (famoso/poco conocido) ").strip().lower()
    return genero, longitud, libro_reciente, tipo_autor

def crear_consulta(genero, longitud, libro_reciente, tipo_autor):
    longitud_filter = {
        "corto": "FILTER (?pages <= 200)",
        "medio": "FILTER (?pages > 200 && ?pages <= 400)",
        "largo": "FILTER (?pages > 400)"
    }

    genero_uri = urllib.parse.quote(genero)
    autor_filter = """
            BIND(((?AuthorRating - {float(min_max[0]['minRating']['value'])}) / ({float(min_max[0]['maxRating']['value'])} - {float(min_max[0]['minRating']['value'])}) * 0.5 + 
            (?count - {float(min_max[0]['minCount']['value'])}) / ({float(min_max[0]['maxCount']['value'])} - {float(min_max[0]['minCount']['value'])}) * 0.5) AS ?popularity)
            """
    if tipo_autor == "famoso":
        autor_filter += f"FILTER (?popularity >= {popularity_threashold})"
    elif tipo_autor == "no conocido":
        autor_filter += f"FILTER (?popularity < {popularity_threashold})"
    else:
        autor_filter = ""

    consulta = f"""
    PREFIX ns: <http://example.org/ns#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    SELECT ?book ?title ?pages ?authorName ?count ?AuthorRating ?popularity (AVG(?rating) AS ?avgRating) (COUNT(?similarBook) AS ?similarityCount)
        WHERE {{
            ?book rdf:type ns:Book .
            ?book ns:title ?title .
        
            # Filtrar por género preferido
            ?book ns:hasGenre ?genreAssignment .
            ?genreAssignment ns:genreType ns:{genero_uri} .
        
            # Filtrar por longitud del libro
            ?book ns:hasPages ?pages .
            {longitud_filter[longitud]}

            # Para calcular el rating medio del libro y ordenarlo de esa forma
            ?readInstance ns:aboutBook ?book .
            ?readInstance ns:hasRating ?rating .
            FILTER(datatype(?rating) = xsd:integer) 
            
            # Incluir autor favorito si está disponible
            OPTIONAL {{
                ?book ns:writtenBy ?author .
                ?author ns:hasName ?authorName .
            }}
            OPTIONAL {{
                # Autor famoso o no
                ?author ns:hasAvgRating ?AuthorRating .
                ?author ns:ratedBy ?count .
            }}
            # Incluir libros similares a uno que el usuario ha leído recientemente
            OPTIONAL {{
                ?book ns:isSimilarTo ?similarBook .
                ?similarBook ns:title ?similarToTitle .
                ?similarBook ns:title "{libro_reciente}" .
            }}
            {autor_filter}   
        }}
        GROUP BY ?book ?title ?pages ?authorName ?count ?AuthorRating ?popularity
        ORDER BY desc(?similarityCount) desc(?avgRating)

    """
    return consulta


endpoint_url = "http://localhost:7200/repositories/new_project"  # Cambia esto según tu configuración

genero, longitud, libro_reciente, tipo_autor = pedir_preferencias()
consulta = crear_consulta(genero, longitud, libro_reciente, tipo_autor)
resultados = ejecutar_consulta(endpoint_url, consulta)


In [87]:

from colorama import Fore, Style, init
def imprimir_resultados(resultados):
    init(autoreset=True)  # Inicializar colorama para restablecer los colores automáticamente
    
    print(f"{'Title':<50} {'Pages':<10} {'Author':<30}")
    print("="*135)
    
    similar_found = False
    for idx, resultado in enumerate(resultados):
        title = resultado["title"]["value"]
        pages = resultado["pages"]["value"] if "pages" in resultado else "N/A"
        author_name = resultado["authorName"]["value"] if "authorName" in resultado else "N/A"
        avg_rating = resultado["avgRating"]["value"] if "avgRating" in resultado else "N/A"
        popularity = resultado["popularity"]["value"] if "popularity" in resultado else "N/A"
        similarity_count = resultado["similarityCount"]["value"] if "similarityCount" in resultado else "N/A"
        
        color = Style.RESET_ALL
        if int(similarity_count) > 0 and not similar_found:
            color = Fore.GREEN
            similar_found = True
        elif idx < 10:
            color = Fore.BLUE
        
        print(f"{color}{title:<50} {pages:<10} {author_name:<30}")


In [88]:
imprimir_resultados(resultados["results"]["bindings"][:50])

Title                                              Pages      Author                        
First Light                                        286        Charles Baxter                
Welcome to the Monkey House (Dell #9478)           308        Kurt Vonnegut Jr.             
Fountain of Dreams (Akora, #4)                     384        Josie Litton                  
A Walk Through the Fire (Sharon McCone, #19)       293        Marcia Muller                 
Cover the Butter                                   368        Carrie Kabak                  
Lo's Diary                                         336        Pia Pera                      
The Boy Knight: A Tale of the Crusades             272        G.A. Henty                    
Anne of the Island                                 240        L.M. Montgomery               
The Razor's Edge                                   246        W. Somerset Maugham           
Blue Light                                         400        Walter M